# Public Transport Data Analysis
This notebook contains the code to 
- Analyze & Aggregate the downloaded data
- Corridor Analysis for different Bybanen lines

## Install packages
Use pip install or poetry install to install the packages.
Run the following commands from the terminal open in root directory of this repository.
```
poetry config virtualenvs.in-project true
poetry install
```
These will create a virtual environment and install the packages specified in the pyproject.toml file. This virtual environment can be used to run this notebook.

## Import Libraries

In [ ]:
 # Import the required packeages
import os
import urllib3
urllib3.disable_warnings()
import pandas as pd
import matplotlib.pyplot as plt
import dask.dataframe as dd
from sqlalchemy import create_engine
import geopandas as gp
from keplergl import KeplerGl

## Analyze & Aggregate the Downloaded data

In [ ]:
 # Reading the consistent format data
raw_df = dd.read_parquet('drive/MyDrive/Trips_pq/*.parquet')

In [ ]:
# Defining main columns
imp_cols = ['tripKey','operatingDate','timeKey','lineKey', 'stopKey','sequenceInJourney','onboard','enteredIn','enteredOut','directionCode','allowedOnboard',\
    'realisticCapacity','isSchoolVacation', 'isPublicHoliday']

In [ ]:
df = raw_df[imp_cols].compute()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
# Changing datatype of column operatingDate from object to date
df['operatingDate'] = pd.to_datetime(df['operatingDate'])

In [ ]:
 # Creating hour column 
df['hour'] = df['timeKey'].apply(lambda x:int(x/100))

In [ ]:
# Creating day column
df['day'] = df['operatingDate'].dt.dayofweek

In [ ]:
df = df.sort_values(by=['tripKey','sequenceInJourney'])

In [ ]:
df['endStopKey'] = df.groupby(['tripKey'])['stopKey'].shift(-1).fillna(0).astype(int)

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df

In [ ]:
example_trip_key = '201910010010000601'
df.loc[df['tripKey']==example_trip_key,:]

In [ ]:
#Aggregating two months data with all required columns
agg_df = pd.DataFrame(df.groupby(['lineKey', 'directionCode', 'isSchoolVacation', 'isPublicHoliday','operatingDate', 'sequenceInJourney','day', 'hour',\
     'stopKey', 'endStopKey']).agg({'enteredIn': 'sum', 'enteredOut':'sum', 'onboard':'sum', 'realisticCapacity':'mean', 'allowedOnboard':'mean',\
          'tripKey': 'count'}).reset_index())
agg_df

In [ ]:
# Changing the datatype of the columns 
agg_df[['enteredIn', 'enteredOut','onboard']] = agg_df[['enteredIn', 'enteredOut','onboard']].astype(int)
# Rounding the columns realisticCapacity and allowedOnboard 
agg_df[['realisticCapacity', 'allowedOnboard']] = agg_df[['realisticCapacity', 'allowedOnboard']].round(1)
# Renaming the columns stopkey and tripKey
agg_df = agg_df.rename(columns={"tripKey": "totalTrip", "stopKey": "startStopKey"})

In [ ]:
agg_df

In [ ]:
# Writing two months aggregated data to csv
agg_df.to_csv('Skyss_OctNov2019_trips.csv', sep='\t', index=False)

## Corridor Analysis

### Configuration and Setup

In [ ]:
postgres_username_password = os.getenv('POSTGRES_USERNAME_PASSWORD','--')

In [ ]:
# Creating engine for postgres database connection
engine = create_engine(postgres_username_password,pool_recycle=3600);
conn = engine.connect()

In [ ]:
MAX_RADIUS = 500

### Read the Geometry Data from Postgres

In [ ]:
# Query for stop points within given radius
bybanen_query = f'''SELECT *, ST_AsText(
   ST_PointN(
	  geom,
	  generate_series(1, ST_NPoints(geom))
	   )) as geom_point, ST_Buffer(  
    road.geom::geography, 
    {MAX_RADIUS})::geometry as geom_ring
FROM (select * from bybanen_lines where (route = 'bybanen-1') or (route = 'bybanen-5')) road, (select ssp.*, ST_MakePoint(longitude,latitude) \
    as geom1 from public.skyss_stop_points ssp ) poi
WHERE ST_DWithin(road.geom::geography, poi.geom1::geography, {MAX_RADIUS});'''

In [ ]:
geo_bybanen_df = pd.read_sql(bybanen_query, conn)

In [ ]:
# Converting dataframe to geo dataframe (line geometry)
geo_line_bybanen = gp.GeoDataFrame.from_postgis(bybanen_query,conn,geom_col='geom')
# Converting dataframe to geo dataframe (stop geometry)
geo_point_bybanen = gp.GeoDataFrame.from_postgis(bybanen_query,conn,geom_col='geom1')
# Converting dataframe to geo dataframe (ring geometry)
geo_ring_bybanen = gp.GeoDataFrame.from_postgis(bybanen_query,conn,geom_col='geom_ring')

In [ ]:
# Trips data
trip_df = agg_df

### Visualize the Data

In [ ]:
# Plotting data 
fig, ax = plt.subplots(figsize=(20, 15))
geo_line_bybanen.plot(ax=ax,column="name")
geo_point_bybanen.plot(ax=ax)
#geo_ring_bybanen.plot(ax=ax)

### Plotting in Kepler MAP

In [ ]:
# Writing both geo files into shape files
geo_line_bybanen.to_file('bybanen_geo_line',driver ='ESRI Shapefile')
geo_point_bybanen.to_file('bybanen_geo_point',driver ='ESRI Shapefile')
geo_ring_bybanen.to_file('bybanen_geo_ring',driver ='ESRI Shapefile')

In [ ]:
# Reading both the shape files in geopandas
df_line = gp.read_file("bybanen_geo_line")
df_point = gp.read_file("bybanen_geo_point")
df_ring = gp.read_file("bybanen_geo_ring")

In [ ]:
# Loading an empty map
map_1 = KeplerGl(height=1000)

In [ ]:
# adding data to keplergl map
map_1.add_data(data=df_line, name='geo_line')
map_1.add_data(data=df_point, name='geo_point')
map_1.add_data(data=df_ring, name='geo_ring')

In [ ]:
map_1.save_to_html(data = map_1.data, config = map_1.config, file_name = 'map_1.html')

### Merge Geometry Data with the Trips Data

In [ ]:
 # Convert Geometry column into String Type to avoid aggregation errors
geo_line_bybanen['geom'] = geo_line_bybanen['geom'].astype(str)

In [ ]:
# Merging the datasets (without using query)
start_trips = pd.merge(geo_line_bybanen,trip_df, how='inner',left_on=['stopKey'],right_on=['startStopKey'])
end_trips = pd.merge(geo_line_bybanen,trip_df, how='inner',left_on=['stopKey'],right_on=['endStopKey'])

In [ ]:
 all_trips = pd.concat([start_trips, end_trips])

In [ ]:
 all_trips.shape

#### Filtering Trips
Exclude the trips which cross the boundary of MAX_RADIUS, ie. Remove the Trips which has startStopKey or endStopKey outside the list of stopKeys within radius

In [ ]:
geo_flesland_df = geo_line_bybanen[geo_line_bybanen['Line']=='Flesland']

In [ ]:
within_radius_stops_list = list(geo_flesland_df['stopKey'].unique())
len(within_radius_stops_list)

In [ ]:
# Add the stopkey 0 to include the trips data on the last stop
within_radius_stops_list = within_radius_stops_list+[0]

In [ ]:
inner_trips = all_trips[(all_trips['startStopKey'].isin(within_radius_stops_list)) & (all_trips['endStopKey'].isin(within_radius_stops_list))]

In [ ]:
 inner_trips.shape